In [ ]:
import tweepy
import json
import pandas as pd
from datetime import datetime
from datetime import timedelta
import preprocessor as pre
import regex as re

# Set tickers and max tweets
tickers = ['GME','TSLA','TWTR','AMC','SPY','HMHC','DWAC','AMD','SST','AAPL','AMZN','NVDA','TLRY','NFLX','QQQ','PLTR','FB','BABA','VIX','SOFI','TEAM','RBLX','RSX','WISH','OSU']
max_tweets = 10000 #max per ticker

# Set date/time parameters
cur_time_utc = datetime.utcnow().replace(microsecond=0)
until_date = cur_time_utc.strftime("%Y-%m-%d") #"2022-04-03"#"2022-04-25"#
from_date =  cur_time_utc - timedelta(days=1) #"2022-04-02"#"2022-04-24"#
from_date = from_date.strftime("%Y-%m-%d")

# Authentication
consumer_key = "juf1eoH4Gn5ZqCtsuKm7PUdAO"
consumer_secret = "7VX0jDqBKTAjVzvYiKhvW4b2tAebMc38FzmPrpcnILkJRt5tF5"
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)

# Create a wrapper for the Twitter API
api = tweepy.API(auth, wait_on_rate_limit=True)

# Function: preprocess tweet text
def tweetprocess(tweet):
    #https://towardsdatascience.com/basic-tweet-preprocessing-in-python-efd8360d529e
    cleantweet = pre.clean(tweet)
    cleantweet = cleantweet.lower()
    cleantweet = re.sub("\d+", "", cleantweet)
    cleantweet = re.sub(r'[^\w\s]', '', cleantweet)   
    return cleantweet

# Function: identify tweet quality
def tweetquality(user_verified, favorite_count, retweet_count):
    if user_verified == True or favorite_count > 100 or retweet_count > 10:
        return True
    else:
        return False

# Function: twitter search pagination and rate limit handling
def limit_handled(cursor):
   while True:
       try:
           yield cursor.next()
       except StopIteration:
           break
       except tweepy.errors:
           print('Reached rate limit. Sleeping for >15 minutes')
           time.sleep(15 * 61)

# Function: obtain query list from ticker list
def getqueries(tickers): #returns a list of query strings
    queries = []
    for ticker in tickers:
        querylist = []
        querylist.append('$'+ticker)
        othertickers=[]
        [othertickers.append(t) for t in tickers]
        othertickers.remove(ticker)
        for otherticker in othertickers:
            querylist.append(' -$'+otherticker) 
        tickerquery = ''.join([str(q) for q in querylist]) + ' -filter:retweets' #Exclude retweets
        queries.append(tickerquery)
    return queries

# Function: get tweets using tweepy
def get_tweets(query, since_id, until_date, max_tweets):

    #search for tweets using Tweepy
    search = limit_handled(tweepy.Cursor(api.search_tweets #https://docs.tweepy.org/en/stable/api.html#search-tweets
                        ,q=query
                        ,tweet_mode='extended'
                        ,lang='en'
                        ,result_type="recent"
                        ,since_id = since_id
                        ,until=until_date
                        ).items(max_tweets))
    dftweets = pd.DataFrame()
    for tweet in search:
        dftweets = dftweets.append(pd.json_normalize(tweet._json))
    print(query,'\n','# tweets:',len(dftweets))

    # calculate attributes
    try:
        dftweets['full_text_preprocessed'] = dftweets.apply(lambda row : tweetprocess(row['full_text']), axis = 1)
        dftweets['quality'] = dftweets.apply(lambda row : tweetquality(row['user.verified'], row['favorite_count'], row['retweet_count']), axis = 1)
        dftweets['num_cashtags'] = dftweets.apply(lambda row : str(row['entities.symbols']).count('text'), axis = 1)
        dftweets['ticker'] = dftweets.apply(lambda row : query.split()[0], axis = 1)
        dftweets['query_params'] = dftweets.apply(lambda row : 'query:'+query+' since_id:'+str(since_id)+' until_date:'+str(until_date)+' max_tweets:'+str(max_tweets), axis = 1)
        #apply filter
        dftweets = dftweets[dftweets.num_cashtags == 1]
        # return output
        return dftweets[['id','ticker','created_at','full_text_preprocessed','user.verified','favorite_count','retweet_count','quality','entities.symbols','num_cashtags','query_params']]
    except Exception:
        pass
    finally:
        print(' # tweets (filtered):',len(dftweets),'\n')

  

# Find the last tweet id for from_date (need this to filter on from_date)
search_since_id = limit_handled(tweepy.Cursor(api.search_tweets #https://docs.tweepy.org/en/stable/api.html#search-tweets
                        ,q='A'
                        ,tweet_mode='extended'
                        ,lang='en'
                        ,result_type="recent"
                        ,until=from_date
                        ).items(1))
since_id  = [tweet._json['id'] for tweet in search_since_id][0]

#Get the tweets
queries = getqueries(tickers)
dftweets_tofile = pd.DataFrame()
for query in queries:
    dftweets_tofile = dftweets_tofile.append(get_tweets(query, since_id, until_date, max_tweets))

# Write the tweets to csv
#cur_time_utc = datetime.utcnow().replace(microsecond=0)
#file_path = 'tweepy_output/twitter '+from_date.strftime('%Y-%m-%d')+'.csv'
#dftweets_tofile.to_csv(file_path, index=False) #,line_terminator = '\r')


$GME -$TSLA -$TWTR -$AMC -$SPY -$HMHC -$DWAC -$AMD -$SST -$AAPL -$AMZN -$NVDA -$TLRY -$NFLX -$QQQ -$PLTR -$FB -$BABA -$VIX -$SOFI -$TEAM -$RBLX -$RSX -$WISH -$OSU -filter:retweets 
 # tweets: 523
 # tweets (filtered): 429 



KeyboardInterrupt: 

In [ ]:
# Write the tweets to csv
cur_time_utc = datetime.utcnow().replace(microsecond=0)
file_path = 'tweepy_output/twitter '+(cur_time_utc - timedelta(days=1)).strftime('%Y-%m-%d')+'.csv'
dftweets_tofile.to_csv(file_path, index=False) #,line_terminator = '\r')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4bd80a34-af3c-4211-a433-b5e963efdc38' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>